In [5]:
def create_master_table(trans, item, no_categories, store):
    """
    Creating the master table
    
    Args:
        trans: Pandas DataFrame
            Transactions data
        item: Pandas DataFrame
            Items information
        no_categories: Number of item categories to be selected    
        store: Pandas DataFrame
            Store information
    
    Returns:
        Master Table: Pandas DataFrame
    """
    
    # Identify fast-moving categories
    fast_moving_category = identify_fast_moving_category(trans, item, no_categories)
    
    # Process the data for master table creation
    trans_weekly = get_weekly_sales(item, trans)
    primary_keys = create_primary_keys(trans_weekly)
    weekly_sales = identify_weekly_sales(trans_weekly, primary_keys)
    target_variable = create_target_variable(weekly_sales)
    sales_related_features = get_sales_related_features(target_variable)
    time_related_features = get_time_related_features(target_variable)

    # Merge data into the master table
    master_table = (
        primary_keys.merge(target_variable, on=["week", "outlet_code", "item_department"], how="left")
        .merge(sales_related_features, on=["week", "outlet_code", "item_department"], how="left")
        .merge(time_related_features, on=["week", "outlet_code", "item_department"], how="left")
        .merge(store, on=["outlet_code"], how="left")
        .merge(fast_moving_category, on=["outlet_code", "item_department"], how="left")  # Add fast_moving_category
        .fillna(0)
    )
    # If you want to prioritize specific versions, drop columns explicitly:
    master_table = master_table.drop(columns=['total_sales_qty_y'], errors='ignore')

    # Rename the retained column for clarity (optional)
    master_table.rename(columns={'total_sales_qty_x': 'total_sales_qty'}, inplace=True)
    
    return master_table